# SEIR Optimization

Basado en pygmo:
https://esa.github.io/pygmo2/

In [1]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, '../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pygmo as pg
import pandas as pd
from time import time

from class_SEIR import SEIR
from importdata import ImportData as importdata
from Quarantine import Quarantine

## Create Pygmo Optimization Object

In [2]:
class SEIROptimization:
    def __init__(self,Ir,tr,I_ac_r,I_ac_tr,tsim,alpha,population,bounds):
        self.Ir = Ir
        self.tr = tr
        self.I_ac_r = I_ac_r
        self.I_ac_tr = I_ac_tr
        self.tsim = tsim        
        self.alpha = alpha
        self.population = population        
        self.bounds = bounds
    def fitness(self,x):        
        sol = SEIR(tsim=self.tsim,alpha=self.alpha,beta=x[0],mu=x[1],k=x[2],I=self.Ir[0],I_ac=self.I_ac_r[0],population=self.population)
        sol.integr_sci(0,tsim,0.1)                
        idx=np.searchsorted(sol.t,self.tr)
        res = LA.norm(self.Ir-sol.I[idx])
        return([res])

    def get_bounds(self):
        return(self.bounds)

    def set_bounds(self,bounds):
        self.bounds = bounds
        return(self.bounds)


## Import Data

In [3]:
# Import data: 
tstate = '13'
initdate = datetime(2020,5,15)

Ir,tr,Ir_dates = importdata.importActiveInfected(tstate = tstate, initdate = initdate)
I_ac_r,I_ac_tr,I_ac_dates = importdata.importAcumulatedInfected(tstate = tstate, initdate = initdate)
population = importdata.importPopulation(tstate = tstate)


Infectados Activos
Infectados Acumulados
Import Population


## Optimization Params

In [4]:
tsim = 1000
mob = 0.6
iqt = 0
alpha = Quarantine(mob,iqt=iqt).alpha

# Params to find
# beta,mu,k
lb=[0.01,0.1, 0]
ub=[   1,  2,30]
bounds = [lb,ub]

## Run Optimization

In [5]:
opti = SEIROptimization(Ir=Ir,tr=tr,I_ac_r=I_ac_r,I_ac_tr=I_ac_tr,tsim=tsim,alpha=alpha,population=population,bounds=bounds)
algo = pg.algorithm(pg.pso(gen = 20))
pop = pg.population(opti,50)

In [6]:
t0 = time()
pop = algo.evolve(pop)
t1 = time()
print('Optimization takes %f seconds' %(t1-t0))
print('Fitness Value')
print(pop.champion_f)
print('Optimal arguments')
print(pop.champion_x)

Optimization takes 4.012479 seconds
Fitness Value
[24150.30607111]
Optimal arguments
[0.10127714 0.63360865 0.58954565]


In [7]:
pop.champion_x

array([0.10127714, 0.63360865, 0.58954565])

In [8]:
test_out = SEIR(tsim=tsim,alpha=1,beta=0.4,mu=1,k=1,I=Ir[0],I_ac=I_ac_r[0],population=1000)

In [9]:
test_out.beta

0.4